In [1]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
import glob
import os
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\ajadon1\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [104]:
# Import Dataset

path = "C:\\Users\\ajadon1\\Desktop\\projects\\featureEngineering\\topic_modelling\\data\\"  #path to folder 

all_files = glob.glob(os.path.join(path, "*aug.csv"))   
# advisable to use os.path.join as this makes concatenation OS independent
colsToKeep = ["MEDICARECLAIMNUMBER","PYRESOLVEDTIMESTAMP_CST","WRAPUPCOMMENTS"]

df_from_each_file = (pd.read_csv(f,encoding = "ISO-8859-1")[colsToKeep].dropna() for f in all_files)
df   = pd.concat(df_from_each_file, ignore_index=True)

#### remove duplicacy
df['PYRESOLVEDTIMESTAMP_CST'] = df.PYRESOLVEDTIMESTAMP_CST.str.slice(0, 9)
df = df.drop_duplicates(keep= 'first').reset_index(drop=True)

C:\Users\ajadon1\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: DtypeWarning: Columns (3,7,8,9,11,12,17,24,25,26,27,28,29,30,31,32,33,35,36,37,38,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\ajadon1\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: DtypeWarning: Columns (24,26,28,43) have mixed types. Specify dtype option on import or set low_memory=False.
  # Remove the CWD from sys.path while we load stuff.


In [115]:
df

,MEDICARECLAIMNUMBER,PYRESOLVEDTIMESTAMP_CST,WRAPUPCOMMENTS
0,259928206A,01-AUG-17,closing April 2017 intent
1,557682433A,01-AUG-17,j
2,522198052A,01-AUG-17,for crs research purposes only
3,436684705B6,01-AUG-17,for crs research purposes only
4,358362750T,01-AUG-17,CRS OUTBOUND. Contacted member and documented ...
5,527645256D,01-AUG-17,closing intent from 4/17 -
6,524268514A,01-AUG-17,closing intent from April 2017
7,192309740A,01-AUG-17,checked back on member
8,500526026A,01-AUG-17,Checking in call.
9,A265067938,01-AUG-17,COMPLETE


In [118]:
df1 = df.sort_values(['MEDICARECLAIMNUMBER', 'PYRESOLVEDTIMESTAMP_CST']) \
.groupby(['MEDICARECLAIMNUMBER', 'PYRESOLVEDTIMESTAMP_CST'], sort=False).WRAPUPCOMMENTS \
       .apply(','.join).reset_index(name='GRP_WRAPUPCOMMENTS')
    
    
df1 

,MEDICARECLAIMNUMBER,PYRESOLVEDTIMESTAMP_CST,GRP_WRAPUPCOMMENTS
0,001013183A,04-AUG-17,member satisfied
1,001029695A,01-AUG-17,"vai,New EE LIDIYA P WOODWARD ID 903697478-1 ca..."
2,001189078A,01-AUG-17,Verified member's contact information and educ...
3,001201670A,01-AUG-17,the member called in to see about what he owes...
4,001226750A,03-AUG-17,Member's spouse passed away this morning. (8/3...
5,001260677A,01-AUG-17,vai
6,001265985A,04-AUG-17,JCI-Script completed. Member had no further qu...
7,001268165A,01-AUG-17,HIPPA Verified/\nMember Name: WATSON MARY\nMem...
8,001268165A,03-AUG-17,CRS Outbound. Documented additional issue pres...
9,001281517A,01-AUG-17,VAI\nmember upset that a test was denied (A024...


In [112]:
df1

,MEDICARECLAIMNUMBER,PYRESOLVEDTIMESTAMP_CST,new_seq
0,001013183A,04-AUG-17,member satisfied
1,001029695A,01-AUG-17,"vai,New EE LIDIYA P WOODWARD ID 903697478-1 ca..."
2,001189078A,01-AUG-17,Verified member's contact information and educ...
3,001201670A,01-AUG-17,the member called in to see about what he owes...
4,001226750A,03-AUG-17,Member's spouse passed away this morning. (8/3...
5,001260677A,01-AUG-17,vai
6,001265985A,04-AUG-17,JCI-Script completed. Member had no further qu...
7,001268165A,01-AUG-17,HIPPA Verified/\nMember Name: WATSON MARY\nMem...
8,001268165A,03-AUG-17,CRS Outbound. Documented additional issue pres...
9,001281517A,01-AUG-17,VAI\nmember upset that a test was denied (A024...


In [90]:

aa = df.drop_duplicates(keep= 'first')
aa.drop_duplicates(keep= 'first')

,MEDICARECLAIMNUMBER,PYRESOLVEDTIMESTAMP_CST,WRAPUPCOMMENTS
0,259928206A,01-AUG-17,closing April 2017 intent
1,557682433A,01-AUG-17,j
2,522198052A,01-AUG-17,for crs research purposes only
3,436684705B6,01-AUG-17,for crs research purposes only
4,358362750T,01-AUG-17,CRS OUTBOUND. Contacted member and documented ...
6,527645256D,01-AUG-17,closing intent from 4/17 -
7,524268514A,01-AUG-17,closing intent from April 2017
8,192309740A,01-AUG-17,checked back on member
9,500526026A,01-AUG-17,Checking in call.
17,A265067938,01-AUG-17,COMPLETE
